In [1]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
print("="*80)
print("🔮 SCRIPT DE INFERENCIA - PREDICCIÓN PARA 2025-2S")
print("="*80)

🔮 SCRIPT DE INFERENCIA - PREDICCIÓN PARA 2025-2S


In [3]:
# Cargar modelos y encoders
print("\n📂 Cargando modelos y configuración...")
label_encoders = joblib.load('models/label_encoders.pkl')
feature_info = joblib.load('models/feature_info.pkl')


📂 Cargando modelos y configuración...


In [4]:
# Modelos disponibles
modelos_disponibles = {
    '1': ('Random Forest', 'models/random_forest_model.pkl'),
    '2': ('SVM', 'models/svm_model.pkl'),
    '3': ('Regresión Logística', 'models/logistic_regression_model.pkl'),
    '4': ('XGBoost', 'models/xgboost_model.pkl')
}

print("\n🤖 Modelos disponibles:")
for key, (nombre, _) in modelos_disponibles.items():
    print(f"  {key}. {nombre}")


🤖 Modelos disponibles:
  1. Random Forest
  2. SVM
  3. Regresión Logística
  4. XGBoost


In [5]:
# Seleccionar modelo
modelo_seleccionado = input("\n👉 Selecciona el modelo a usar (1-4): ").strip()

if modelo_seleccionado not in modelos_disponibles:
    print("❌ Opción inválida. Usando Random Forest por defecto.")
    modelo_seleccionado = '1'

nombre_modelo, ruta_modelo = modelos_disponibles[modelo_seleccionado]
print(f"\n✅ Modelo seleccionado: {nombre_modelo}")

❌ Opción inválida. Usando Random Forest por defecto.

✅ Modelo seleccionado: Random Forest


In [ ]:
# Cargar modelo
modelo = joblib.load(ruta_modelo)
print(f"✓ Modelo cargado desde '{ruta_modelo}'")

# Cargar datos de inferencia
print("\n📊 Cargando datos de inferencia...")
try:
    df_inferencia = pd.read_csv('models/datos_inferencia_2025_2S.csv')
    print(f"✓ Datos cargados: {len(df_inferencia)} registros")
except FileNotFoundError:
    print("❌ No se encontró el archivo 'models/datos_inferencia_2025_2S.csv'")
    print("   Asegúrate de haber ejecutado el script de entrenamiento primero.")
    exit()

In [ ]:
# Verificar que tengamos los datos correctos
if len(df_inferencia) == 0:
    print("❌ No hay datos disponibles para inferencia.")
    exit()

# Extraer features
X_columns = feature_info['X_columns']
X_inferencia = df_inferencia[X_columns]

print(f"\n🔮 Realizando predicciones...")
# Predicciones
predicciones = modelo.predict(X_inferencia)
probabilidades = modelo.predict_proba(X_inferencia)[:, 1]  # Probabilidad de reprobar

# Agregar predicciones al dataframe
df_inferencia['PREDICCION'] = predicciones
df_inferencia['PROB_REPROBAR'] = probabilidades
df_inferencia['PREDICCION_TEXTO'] = df_inferencia['PREDICCION'].apply(
    lambda x: 'REPROBARÁ' if x == 1 else 'APROBARÁ'
)

# Resultados
print(f"\n{'='*80}")
print(f"📊 RESULTADOS DE INFERENCIA")
print(f"{'='*80}")

print(f"\n📈 Distribución de predicciones:")
print(df_inferencia['PREDICCION_TEXTO'].value_counts())
print(f"\n  • Porcentaje predicho de reprobación: {(predicciones == 1).mean()*100:.2f}%")
print(f"  • Porcentaje predicho de aprobación: {(predicciones == 0).mean()*100:.2f}%")

print(f"\n📊 Estadísticas de probabilidades:")
print(f"  • Probabilidad promedio de reprobar: {probabilidades.mean():.4f}")
print(f"  • Probabilidad mínima: {probabilidades.min():.4f}")
print(f"  • Probabilidad máxima: {probabilidades.max():.4f}")
print(f"  • Desviación estándar: {probabilidades.std():.4f}")

# Estudiantes en riesgo (probabilidad > 0.5)
estudiantes_riesgo = df_inferencia[df_inferencia['PROB_REPROBAR'] > 0.5]
print(f"\n⚠️ Estudiantes en ALTO RIESGO de reprobar (prob > 0.5): {len(estudiantes_riesgo)}")

# Estudiantes en riesgo moderado (0.3 < prob <= 0.5)
estudiantes_riesgo_moderado = df_inferencia[
    (df_inferencia['PROB_REPROBAR'] > 0.3) & (df_inferencia['PROB_REPROBAR'] <= 0.5)
]
print(f"⚠️ Estudiantes en RIESGO MODERADO (0.3 < prob <= 0.5): {len(estudiantes_riesgo_moderado)}")

# Guardar resultados
output_file = f'models/predicciones_2025_2S_{nombre_modelo.replace(" ", "_").lower()}.csv'
df_inferencia.to_csv(output_file, index=False)
print(f"\n💾 Predicciones guardadas en '{output_file}'")

# Mostrar algunos ejemplos
print(f"\n📋 EJEMPLOS DE PREDICCIONES (primeros 10 registros):")
print("="*80)
columnas_mostrar = ['COD_ESTUDIANTE', 'COD_MATERIA_ACAD_MO', 'PREDICCION_TEXTO', 'PROB_REPROBAR']
if 'COD_ESTUDIANTE' in df_inferencia.columns:
    print(df_inferencia[columnas_mostrar].head(10).to_string(index=False))
else:
    print(df_inferencia[['PREDICCION_TEXTO', 'PROB_REPROBAR']].head(10).to_string(index=False))

# Si tenemos las etiquetas reales, calcular métricas
if 'REPROBO' in df_inferencia.columns:
    from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
    
    y_real = df_inferencia['REPROBO']
    
    print(f"\n{'='*80}")
    print(f"📊 EVALUACIÓN DEL MODELO EN DATOS DE INFERENCIA (2025-2S)")
    print(f"{'='*80}")
    
    accuracy = accuracy_score(y_real, predicciones)
    roc_auc = roc_auc_score(y_real, probabilidades)
    
    print(f"\n✅ Métricas de rendimiento:")
    print(f"  • Accuracy: {accuracy:.4f}")
    print(f"  • ROC-AUC: {roc_auc:.4f}")
    
    print(f"\n📋 Classification Report:")
    print(classification_report(y_real, predicciones, target_names=['Aprobado', 'Reprobado']))
    
    cm = confusion_matrix(y_real, predicciones)
    print(f"\n📊 Matriz de Confusión:")
    print(f"  [[TN={cm[0,0]}, FP={cm[0,1]}],")
    print(f"   [FN={cm[1,0]}, TP={cm[1,1]}]]")
    
    print(f"\n  Interpretación:")
    print(f"  • Verdaderos Negativos (TN): {cm[0,0]} - Correctamente predichos como APROBARÁ")
    print(f"  • Falsos Positivos (FP): {cm[0,1]} - Incorrectamente predichos como REPROBARÁ")
    print(f"  • Falsos Negativos (FN): {cm[1,0]} - Incorrectamente predichos como APROBARÁ")
    print(f"  • Verdaderos Positivos (TP): {cm[1,1]} - Correctamente predichos como REPROBARÁ")

print(f"\n{'='*80}")
print(f"✅ INFERENCIA COMPLETADA")
print(f"{'='*80}")